# Image creation

# `docker build` - build image{#sec-docker_build_command}

This is the command used to create a new docker image.

Basic arguments:

- the last mandatory argument sets the build directory - you should use `.` to set the current directory.
- `-t` - set the name and tag of the image.

In [2]:
%%bash
cd build_command_files
docker build -t custom_ubuntu:test .
docker images | grep custom_ubuntu
docker rmi custom_ubuntu:test

#1 [internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [internal] load build definition from dockerfile
#2 transferring dockerfile: 55B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:latest
#3 DONE 0.0s

#4 [1/1] FROM docker.io/library/ubuntu:latest
#4 CACHED

#5 exporting to image
#5 exporting layers done
#5 writing image sha256:8173c380186319e1e49d54260624b8b78635789ab603389d63c65e828e150c8f done
#5 naming to docker.io/library/custom_ubuntu:test done
#5 DONE 0.0s


custom_ubuntu              test      8173c3801863   3 months ago    77.8MB
Untagged: custom_ubuntu:test
Deleted: sha256:8173c380186319e1e49d54260624b8b78635789ab603389d63c65e828e150c8f


### `-f` - allow to chose the dockerfile

You should see a difference between building directory and dockerfile. The `dockerfile` is only taken from the building directory if it is not specified in the `-f` option. But the files used for the build will in any case come from the build directory.

So let's try some experiments with these details. I have prepared some folders for experiments with these options, in the following cell I show tree and file contents.

In [13]:
%%bash
cd build_command_files/f_option_examples
tree

echo
echo
echo

for file in $(find -type f | grep -v "\/\."); do
    echo ""
    echo "=====File: $file====="
    cat $file
    echo ""
done

.
├── dockerfile
├── script.sh
├── specialdockerfile
└── test_folder
    ├── dockerfile
    └── script.sh

1 directory, 5 files




=====File: ./dockerfile=====
FROM ubuntu:latest
COPY script.sh script.sh
RUN echo "\necho message from basic dockerfile" >> script.sh

=====File: ./script.sh=====
echo "It's a script.sh from the run folder"

=====File: ./specialdockerfile=====
FROM ubuntu:latest
COPY script.sh script.sh
RUN echo "\necho message from specialdockerfile" >> script.sh

=====File: ./test_folder/dockerfile=====
FROM ubuntu:latest
COPY script.sh script.sh
RUN echo "\necho message from test_folder/dockerfile" >> script.sh

=====File: ./test_folder/script.sh=====
echo "It's a script from test_folder"


We have:

- scripts:
    - from run folder - prints `"It's a script.sh from the run folder"`;
    - from test_folder - prints `"It's a script from test_folder"`;
- dockerfiles:
    - basic `dockerfile` from run directory - adds to printing script line `"message from basic dockerfile"`;
    - `specisaldocker` file from run dicrecotry but with special name - adds to printing script line `"message from specialdockerfile"`;
    - `dockerfile` from `test_folder` directory - adds to printing script line `"message from test_folder/dockerfile"`;
    - `specialdockerfile` from `test_folder` directory - adds to printing script line `"message from test_folder/specialdockerfile"`;

This way we can always tell from the output of `script.sh` from the container in which folder and from which dockerfile the image was built. Let's try different options:

- The most basic case no `-f` option and `.` as build directory - will lead to using docker file from run directory and `script.sh` was copiet from run dicrecory as well;

In [16]:
%%bash
cd build_command_files/f_option_examples
docker build -t test_ubuntu . &> /dev/null
docker run --rm -itd \
    --name test_ubuntu \
    test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh

docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====message from container=====
It's a script.sh from the run folder
message from basic dockerfile


- If you do not set the `-f` option but specify the build folder as `test_folder` everythig will be taken from `test_folder`;

In [17]:
%%bash
cd build_command_files/f_option_examples
docker build -t test_ubuntu test_folder &> /dev/null
docker run --rm -itd \
    --name test_ubuntu \
    test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh

docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====message from container=====
It's a script from test_folder
message from test_folder/dockerfile


- If you set `-f specialdockerfile` and specify the build folder as `test_folder` - everything is obvious;

In [19]:
%%bash
cd build_command_files/f_option_examples
docker build \
    -t test_ubuntu \
    -f specialdockerfile \
    test_folder &> /dev/null
docker run --rm -itd \
    --name test_ubuntu \
    test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh

docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====message from container=====
It's a script from test_folder
message from specialdockerfile


- Bit exotic case - `dockerfile` from `test_folder`, but build directory is run directory;

In [20]:
%%bash
cd build_command_files/f_option_examples
docker build \
    -t test_ubuntu \
    -f test_folder/dockerfile \
    . &> /dev/null
docker run --rm -itd \
    --name test_ubuntu \
    test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh

docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====message from container=====
It's a script.sh from the run folder
message from test_folder/dockerfile


**Note** even if you have specified a folder as some assembly folder, you must still pass in the `-f` parameter the path relative to the run folder.

The next two points show the difference:

- Here I set building directory as `test_folder` but `-f specialdockerfile` - `specialdockerfile` from run folder will be used; 

In [22]:
%%bash
cd build_command_files/f_option_examples
docker build \
    -t test_ubuntu \
    -f specialdockerfile \
    test_folder &> /dev/null
docker run --rm -itd \
    --name test_ubuntu \
    test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh

docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====message from container=====
It's a script from test_folder
message from specialdockerfile


- Here I set building directory as `test_folder` but `-f test_folder/specialdockerfile` - `specialdockerfile` from `test_folder` will be used; 

In [23]:
%%bash
cd build_command_files/f_option_examples
docker build \
    -t test_ubuntu \
    -f test_folder/specialdockerfile \
    test_folder &> /dev/null
docker run --rm -itd \
    --name test_ubuntu \
    test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh

docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====message from container=====
It's a script from test_folder
message from test_folder/specialdockerfile


# .dockerignore - select files ignored during image building

In docker ignore, you should specify files to be ignored when building the image.

For example lets try to build image with followig paremeters:

In [13]:
%%bash
cd dockerignore
echo "=====dockerfile====="
cat dockerfile1
echo
echo "=====.dockerignore====="
cat .dockerignore

=====dockerfile=====
FROM ubuntu
COPY test_file test_file
=====.dockerignore=====
test_file
test_folder/banned_file


So in dockerfile I try to copy `test_file` into image, but in docker ignore I bun this file. At the next cell, I'm trying to build such image - and getting error.

In [12]:
%%bash
cd dockerignore
docker build -t test_image -f dockerfile1 .

#1 [internal] load build definition from dockerfile1
#1 transferring dockerfile: 74B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 transferring context: 50B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:latest
#3 DONE 0.0s

#4 [1/2] FROM docker.io/library/ubuntu
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [2/2] COPY test_file test_file
#6 ERROR: failed to calculate checksum of ref 40880b12-160a-424d-a82b-d59b9594f64c::7cmhtk0ygxakortpvzlpd8pwp: "/test_file": not found
------
 > [2/2] COPY test_file test_file:
------
dockerfile1:2
--------------------
   1 |     FROM ubuntu
   2 | >>> COPY test_file test_file
--------------------
ERROR: failed to solve: failed to compute cache key: failed to calculate checksum of ref 40880b12-160a-424d-a82b-d59b9594f64c::7cmhtk0ygxakortpvzlpd8pwp: "/test_file": not found


CalledProcessError: Command 'b'cd dockerignore\ndocker build -t test_image -f dockerfile1 .\n'' returned non-zero exit status 1.

On the other hand, if I copy an entire folder and only a few files are blocked in the folder, the folder will be copied without those files. The following example show such configuration:

In [24]:
%%bash
cd dockerignore

echo "=====dockerfile====="
cat dockerfile2
echo
echo "=====.dockerignore====="
cat .dockerignore

echo
echo "=====test_folder in host====="
ls test_folder

docker build -t test_image -f dockerfile2 . &> /dev/null

docker run --rm --name test_container -itd test_image &> /dev/null
echo "=====test_folder in container====="
docker exec test_container ls test_folder

docker stop test_container &> /dev/null
docker rmi test_image &> /dev/null

=====dockerfile=====
FROM ubuntu
COPY test_folder test_folder
=====.dockerignore=====
test_file
test_folder/banned_file

=====test_folder in host=====
accepted_file
banned_file
=====test_folder in container=====
accepted_file


Here I copy the whole folder, but the file `banned_file` was mentioned in `.dockerignore`. So when we `ls temp_folder` from container we see only `accepted_file` in folder contents.

# File in container forever

If you copied a file into the container, but literally deleted it by the next instruciton - it will still affect the size of the container.

Following exampe show that fature:
- First I create an image - just copy Ubuntu;
- Second, add the `COPY` instruction for a large file - so the size of the container will be much larger than in the first step;
- Last image add `RUN rm` for the file copied in the previous step, the point is that the image size hasn't changed.

In [10]:
%%bash

dd if=/dev/zero of=file.txt bs=1M count=1000 &> /dev/null

echo "=====just ubuntu copy====="
echo "FROM ubuntu" > test_dockerfile
docker build -f test_dockerfile -t test_image . &> /dev/null
docker images | grep test_image
docker rmi test_image &> /dev/null

echo "=====ubuntu with big copied file====="
echo "COPY file.txt file.txt" >> test_dockerfile
docker build -f test_dockerfile -t test_image . &> /dev/null
docker images | grep test_image
docker rmi test_image &> /dev/null

echo "=====ubuntu with deleted file====="
echo "RUN rm file.txt" >> test_dockerfile
docker build -f test_dockerfile -t test_image . &> /dev/null
docker images | grep test_image
docker rmi test_image &> /dev/null


rm file.txt
rm test_dockerfile

=====just ubuntu copy=====
test_image                 latest    8173c3801863   3 months ago    77.8MB
=====ubuntu with big copied file=====
test_image                 latest    86892a85e309   2 minutes ago   1.13GB
=====ubuntu with deleted file=====
test_image                 latest    97524578efc6   52 seconds ago   1.13GB
